本章开头，主要介绍Excel对象模型和xlwings

1. 学习基础知识， 如何连接工作簿、读写单元格的值
2. 深度学习如何利用转换器和各种选项来处理pandas DataFrame和Numpy数组。
3. 解释xlwings工作原理

## 1.开始使用xlwings
### 1.1 将Excel用作数据查看器



In [5]:
import datetime as dt
import xlwings as xw
import pandas as pd
import numpy as np

In [6]:
df = pd.DataFrame(data=np.random.randn(100,5), columns=[f'Traub {i}' for i in range(1, 6)])
df 

,Traub 1,Traub 2,Traub 3,Traub 4,Traub 5
0,0.716805,0.522042,1.126891,-0.818033,0.383853
1,0.667052,-0.525294,1.316766,1.768725,0.036111
2,-0.583887,1.224307,-2.614621,-0.787321,0.084570
3,0.826711,0.595808,1.789517,0.231150,-0.503082
4,0.976986,-0.397566,1.111736,0.426395,-2.813906
...,...,...,...,...,...
95,-0.012022,0.703707,-0.046439,-0.353314,-0.321509
96,0.807168,0.334828,-0.337283,0.945607,0.897734
97,-0.197056,0.677489,-2.627688,0.072414,-1.971709
98,-1.465213,1.870936,-1.269265,-0.188267,0.601951


In [7]:
xw.view(df) #view相当于将数据复制到一张工作表的A1单元格上。 此处直接打开了一个excel

### 1.2 Excel对象模型

In [8]:
# 创建一个空的工作簿， 此处打开了一个excel
book = xw.Book()
book.name

'工作簿2'

In [9]:
book.sheets

Sheets([<Sheet [工作簿2]Sheet1>])

In [10]:
# 通过索引或者名称获取工作表对象
sheet1 = book.sheets[0] # sheet1 = books.sheets['Sheet1']

In [11]:
sheet1.range('A1')

<Range [工作簿2]Sheet1!$A$1>

In [12]:
# 写入值， 试试刷新了excel
sheet1.range('A1').value = [[1,2],[3,4]]
sheet1.range('A4').value = 'Hello!'



In [13]:
# 读取值
sheet1.range('A1:B2').value

[[1.0, 2.0], [3.0, 4.0]]

In [14]:
sheet1.range('A4').value

'Hello!'

In [15]:
# 索引
sheet1.range("A1:B2")[0,0]

<Range [工作簿2]Sheet1!$A$1>

In [16]:
# 切片
sheet1.range('A1:B2')[:,1]

<Range [工作簿2]Sheet1!$B$1:$B$2>

In [17]:
# 除了上述直接使用range来使用sheet对象的range属性， 也可以通过对sheet对象进行索引和切片来获取一个range对象。
# 利用A1表示法可以少让你敲一些字，而使用整个切片可以让Excel工作表看上去更像是Numpy数组

# 单个单元格
sheet1["A1"]

<Range [工作簿2]Sheet1!$A$1>

In [18]:
# 多个单元格 A1表示法
sheet1['A1:B2']

<Range [工作簿2]Sheet1!$A$1:$B$2>

In [19]:
# 单个单元格：索引
sheet1[0,0]

<Range [工作簿2]Sheet1!$A$1>

In [20]:
# 多个单元格： 切片
sheet1[:2,:2]

<Range [工作簿2]Sheet1!$A$1:$B$2>

如何从range对象自底向上得到app对象

In [21]:
sheet1['A1'].sheet.book.app

<App [excel] 4036>

是时候看看如何利用多个Excel实例了。如何想在多个Excel实例中打开同一个工作簿， 或是出于性能方面的原因想要将多个工作簿分发给多个实例， 则需要**显式**地使用app对象。app对象的另一个常见用例是在隐藏的Excel实例中打开工作簿，这样你就可以在后台运行xlwings脚本且同时在Excel中完成其他工作。

In [22]:
# 从打开的工作簿中获取app对象， 并创建一个额外的隐藏的app实例
visble_app = sheet1.book.app
invisible_app = xw.App(visible=False)

In [23]:
[book.name for book in visble_app.books]

['工作簿1', '工作簿2']

In [24]:
[book.name for book in invisible_app.books]

['工作簿3']

In [25]:
# app的键带边隐藏ID(PID)
xw.apps.keys()

[14984, 4036]

In [26]:
xw.apps.active.pid

14984

In [27]:
# 处理隐藏的Excel实例中的工作簿
invisible_book = invisible_app.books[0]
invisible_book.sheets[0]['A1'].value = 'Create by an invisible app'
invisible_book.save('invisible.xlsx')
invisible_app.quit()

### 运行VBA代码

通常在Excel中 我们会构建大量的VBA代码的旧式Excel项目， 那么要将所有的东西转换逻辑，利用Python代码进行重新构建是很难的。 在这种情况下， 我们可通过Python来运行VBA宏。

通过Python调用Excel的接口， 我们不关注VBA内部函数的具体实现，只需要知道需要调用的函数名与输入参数即可。

In [28]:
vba_book = xw.Book('../ori_writer/xl/vba.xlsm')
mysum = vba_book.macro('Module1.MySum')
mysum(5,4)

9.0

In [29]:
# 在excel中弹出对话框
show_msgbox = vba_book.macro('Module1.ShowMsgBox')
show_msgbox("Hello xlwings!")


In [30]:
vba_book.close()

## 2. 转换器、选项和集合

这节我们介绍如何在pandas中完成一些数据处理，并与xw进行配合， 然后我们进一步了解一些参数的含义




### 2.1 处理DataFrame


In [31]:

data = [["Mark", 55, "Italy", 4.5, "Europe"],
        ["John", 33, "USA", 6.7, "America"]]
df = pd.DataFrame(data=data,
                  columns=["name", "age", "country",
                           "score", "continent"],
                  index=[1001, 1000])
df.index.name = "user_id"
df

,name,age,country,score,continent
user_id,,,,,
1001,Mark,55,Italy,4.5,Europe
1000,John,33,USA,6.7,America


In [32]:
sheet1['A6'].value = df

In [33]:
# 如果想去掉列标题或索引, 那么可以设置如下options

sheet1['B10'].options(header=False, index=False).value = df

In [34]:
# 引入第二种方式expand函数， 类似在Excel中右键部分区域数据，然后填充到另一个区域
df2 = sheet1['A6'].expand().options(pd.DataFrame).value

In [35]:
df2

,name,age,country,score,continent
user_id,,,,,
1001.0,Mark,55.0,Italy,4.5,Europe
1000.0,John,33.0,USA,6.7,America


In [36]:
# 修改成整数索引

df2.index = df2.index.astype(int)
df2

,name,age,country,score,continent
user_id,,,,,
1001,Mark,55.0,Italy,4.5,Europe
1000,John,33.0,USA,6.7,America


In [37]:
#设置options中index=False，可以让变成默认索引。类似resetIndex
sheet1['A6'].expand().options(pd.DataFrame, index=False).value


,user_id,name,age,country,score,continent
0,1001.0,Mark,55.0,Italy,4.5,Europe
1,1000.0,John,33.0,USA,6.7,America


### 2.2 转换器和选项

如前所述， xlwings range对象的options方法修改的是读写Excel文件时处理值的方式， 即，只有在你调用range对象的value属性时， options才会进行求值。它的语法如下：

`myrange.options(convert=None, option1=value1, option2=value2, ...).value`

下图展示了convert参数的的选项
|转换器|描述|
|--|--|
|dict|未发生嵌套的简单字典，即{key1:value1, key2:value2,...}的形式|
|np.array|Numpy数组|
|pd.Series|pandas series|
|pd.DataFrame|pandas DataFrame|

下图展示了options的内置选项
|选项|描述|
|--|--|
|empty|默认情况下， 空单元格会被读取为None， 为empty参数提供一个值以改变默认值|
|date|接受一个函数， 该函数会应用来自按日期格式化的单元的值|
|numbers|接受一个函数，该函数会用到数值|
|ndim|维数：ndim可以强制一个区域的值达到某个维度。只能取None、1、2中的一个值， 仅在以列表或Numpy数组读取值时可用|
|transpose|将值转置， 即把行和列兑换|
|index|不好描述|
|header|是否生成列标题|





In [38]:
# 水平方向
sheet1["A1:B1"].value

[1.0, 2.0]

In [39]:
# 垂直方向
sheet1["A1:A2"].value

[1.0, 3.0]

In [40]:
# 水平方向(二维)
sheet1["A1:B1"].options(ndim=2).value

[[1.0, 2.0]]

In [42]:
# 垂直方向（二维）
sheet1["A1:A2"].options(ndim=2).value

[[1.0], [3.0]]

In [43]:
# 使用numpy数组转换器
sheet1["A1:A2"].options(np.array).value

array([1., 3.])

In [44]:
# 使用numpy转换器，保持列方向不变
sheet1["A1:A2"].options(np.array, ndim=2).value

array([[1.],
       [3.]])

In [45]:
# 垂直写入列表
sheet1['D1'].options(transpose=True).value = [100, 200]

上述中说的ndim和numpy中的ndarray类似。 在Excel中， 一列可以是一个二维的【看做每一行是一个组织，一列有多个行，则是二维的】，也可以看做是一维的【一列就是一列】，所以使用ndim能够将其进行转换来看

In [46]:
# 从A13开始写，B13 = NOne, C13 = 1
sheet1['A13'].value = [dt.datetime(2020,1,1), None, 1.0]
sheet1['A13:C13'].value

[datetime.datetime(2020, 1, 1, 0, 0), None, 1.0]

In [47]:
# 使用非默认选项读取
sheet1['A13:C13'].options(empty='NA', 
                          dataset=dt.date,
                          number=int).value

[datetime.datetime(2020, 1, 1, 0, 0), 'NA', 1.0]

### 2.3 图表、图片和已定义名称



In [48]:
# 1. Excel图表

# 使用charts集合中的addd方法来添加一张新的图表并为其设置图表类型和源数据
sheet1["A15"].value = [[None, "North", "South"],
                       ["Last Year", 2, 5],
                       ["This Year", 3, 6]]

In [49]:
chart = sheet1.charts.add(top=sheet1["A19"].top,
                          left=sheet1["A19"].left)
chart.chart_type = "column_clustered"
chart.set_source_data(sheet1["A15"].expand())

In [50]:
# 2. 图片: Matplotlib图像

# 图表数据转DataFrame
df = sheet1['A15'].expand().options(pd.DataFrame).value

In [51]:
df

,North,South
Last Year,2.0,5.0
This Year,3.0,6.0


In [52]:
%matplotlib inline
import matplotlib.pyplot as plt

ax = df.T.plot.bar() #数据转置， 并画图
fig = ax.get_figure() # 获取图片

# 图片发送给excel
plot = sheet1.pictures.add(fig, name="SalesPlot",
                           top=sheet1["H19"].top,
                           left=sheet1["H19"].left)

plot.width, plot.height = plot.width * 0.7, plot.height * 0.7

In [53]:
# 要喜爱那个使用新的图像来更新图片，只需调用update方法并传递拎一个figure对象即可。 
ax = (df + 1).T.plot.bar()
plot = plot.update(ax.get_figure())

上述对图表集合和图片集合都是通过sheet对象，我们也可以通过book对象来访问。

3. 已定义名称

在Excel中，我们通过为区域、公式、常量赋予名称来创建**已定义名称**。为一个区域明明可能是最常见的情况，这种区域被称作**具名区域**。

利用具名区域， 你可以在公式、代码中使用描述性名称而不是抽象地址（A1:B2）来引用一个Excel区域。 

在xlwings中使用这些名称可以让代码更加灵活且更加稳定： 利用具名区域读写值可以在不调整python代码的情况相爱重新组织工作簿。比如： 即使插入新行导致了单元格的移动，但对应的名称仍然引用的是原来的单元格。 自定义名称可以在全局工作簿作用于或局部工作表作用于中设置。 

工作表作用于的优势是在复制工作表时不用担心重复的具名区域发生冲突。

In [55]:
# 默认作用于是工作簿作用于
sheet1["A1:B2"].name = "matrix1"
#对于工作表作用域， 需要再工作表名称前加一个感叹号
sheet1["B10:E11"].name = "Sheet1!matrix2"

# 通过名称访问区域
sheet1['matrix1']

<Range [工作簿2]Sheet1!$A$1:$B$2>

In [56]:
# 如果通过sheet1对象访问名称集合， 则其中只包含工作表作用域的名称
sheet1.names

[<Name 'Sheet1!matrix2': =Sheet1!$B$10:$E$11>]

In [57]:
# 如果通过book对象来访问名称集合， 则其中包含了工作簿和工作表作用域的所有名称

book.names

[<Name 'matrix1': =Sheet1!$A$1:$B$2>, <Name 'Sheet1!matrix2': =Sheet1!$B$10:$E$11>]

In [60]:
# 名称有多种方法和属性， 例如你可以获取对应的range对象
book.names['matrix1'].refers_to_range

<Range [工作簿2]Sheet1!$A$1:$B$2>

In [61]:
# 如果你喜爱那个为常来那个或公式取名， 你可以使用add方法

book.names.add("EURUSD",  "=1.1151")

<Name 'EURUSD': =1.1151>

### 2.4案例研究(再次回顾)：Excel报表

## 3.高级xlwings主题
### 3.1 xlwings的基础
我们可以看出 xlwings其实是基于其他python的资源包来和各个操作系统中的自动化机制通信的

### 3.2 提升性能
想让xlwings脚本有良好的性能，可以从以下入手：

1. 尽可能减少跨应用程序调用
2. 使用原始值
3. 正确设置app的属性也会有一定保住



In [63]:
# 1. 尽可能减少跨应用调用

# 1. 添加新的工作表，写入150个人值以便有事可做

sheet2 = book.sheets.add()
sheet2['A1'].value = np.arange(150).reshape(30,5)



In [64]:
# 1.1 逐个单元格去写
%%time
for cell in sheet2['A1:E30']:
    cell.value += 1

CPU times: total: 0 ns
Wall time: 304 ms


In [65]:
# 1.2 变成numpy 并行去写
%%time
values = sheet2['A1:E30'].options(np.array).value
sheet2['A1:E30'].value = values + 1

CPU times: total: 0 ns
Wall time: 26.4 ms


In [66]:
# 2. 原始值
# 如果处理庞大的单元格区域， 需要跳过xlwings的数据清理阶段来节省时间【续写数据是， xlwings需要遍历每个值，找到对应的位置】，
# 此时使用原始值(raw)作为转换器可以跳过这个过程。使用原始值，则表示不能在使用DataFrame， 需要以嵌套列表或元组的形式提供值
# 同时必须只提供写入区域的完整地址

sheet1['A35:B36'].options('raw').value = [[1,2],[3,4]]

In [ ]:
# 3.app对象的属性
# 根据工作簿内容的不同， 修改app某些属性可能让代码运行的更快
#  在脚本的最后，一定要将app对象的属性还原！！！！！！！

## 3.3 如何弥补缺失的功能

xlwings为大部分常用的Excel命令提供了Python风格的接口，但仍然有很多Excel对象模型的方法和属性没有在xlwings中得到原生的实现。
这样需要访问下xlwings提供的pywin32对象接口了,通过接口COM与Excel对象进行交互访问，同时我们也失去了跨平台兼容性， 比如macOS与windows的对象是不一样的。

遵循如下原则：

1. 检查xlwings的range对象上是否有对应方法可用。在jupyterbook中，输入range对象后面的点之后按下Tab键。 在vscode上， 可用方法会自动显示在提示信息中。
2. 如果找不到所需功能，可以使用api属性或得底层对象。在winfows中， sheet['A1'].api会返回一个pywin32对象
3. 在Excel VBA参考文档中可以查看Excel对象模型。
4. 在windows中大部分时候可以直接在api对象上使用VA方法或实行。




In [ ]:
import sys
if sys.platform.startswith('darwin'):
    sheet1['B10'].api.clear_formats()
elif sys.platform.startswith('win'):
    sheet1['A10'].api.ClearFormats()  # 弹不出方法，因为调用到pywin32对象了